In [1]:
pip install opencv-python mediapipe

In [2]:
import cv2
import mediapipe as mp
import numpy as np

In [3]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [4]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque

# Initialize MediaPipe Holistic and Drawing utilities
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

cap = cv2.VideoCapture(0)

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a) 
    b = np.array(b) 
    c = np.array(c) 

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Variables to keep track of rep count
counter = 0
stage = None

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Real-time Graph Plotting
    plot_x, plot_y = 10, 200
    plot_size = 200
    plot_buffer = deque(maxlen=plot_size)
    plot_threshold = 30  # Threshold for angle plot
    plot_color = (255, 0, 0)

    # Virtual Trainer or Assistant
    virtual_trainer_enabled = True
    virtual_trainer_text = ""
    virtual_trainer_color = (0, 255, 0)

    # Educational Content and Tips
    tips_display_time = 300  # Display tips for 5 seconds
    tips_timer = 0
    display_tips = False
    tips_text = ""

    while cap.isOpened():  # Check if webcam is open or not
        ret, frame = cap.read()  # Our image in the frame

        if not ret:
            break

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.setflags(write=0)  # Set writable to False

        # Make Detections
        results = holistic.process(image)  # Update results with new detections

        # Recolor image back to BGR for rendering
        image.setflags(write=1)  # Set writable to True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Recoloring back to RGB to BGR

        # Extract landmarks for the right arm
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].x,
                        landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].x,
                     landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST.value].x,
                     landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST.value].y]

            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)

            # Visualize angle
            cv2.putText(image, str(angle),
                        tuple(np.multiply(elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA
                        )

            # Update plot buffer
            plot_buffer.append(angle)

            # Draw the angle plot
            plot_image = np.ones((250, 600, 3), dtype=np.uint8) * 255
            cv2.polylines(plot_image, [np.array(list(zip(range(plot_size), plot_buffer)), np.int32)], False, plot_color, 2)
            cv2.putText(plot_image, 'Angle Plot', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.imshow('Angle Plot', plot_image)

            # Virtual Trainer logic
            if virtual_trainer_enabled:
                if angle > 160:
                    virtual_trainer_text = "Lower your arm a bit."
                    virtual_trainer_color = (0, 0, 255)
                elif angle < 30:
                    virtual_trainer_text = "Great posture!"
                    virtual_trainer_color = (0, 255, 0)
                else:
                    virtual_trainer_text = ""
                    virtual_trainer_color = (0, 255, 0)

            # Educational Content and Tips logic
            if tips_timer > 0:
                tips_timer -= 1
            else:
                display_tips = False

            if not display_tips:
                tips_text = "Keep your arm straight to avoid strain."
                display_tips = True
                tips_timer = tips_display_time

            # Display virtual trainer feedback
            cv2.putText(image, virtual_trainer_text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, virtual_trainer_color, 2, cv2.LINE_AA)

            # Display educational tips
            if display_tips:
                cv2.putText(image, tips_text, (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

            # Curl counter logic
            if angle > 120:
                stage = "down"
            if angle < 60 and stage == 'down':
                stage = "up"
                counter += 1
                print(counter)

        # Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_face_mesh.FACEMESH_CONTOURS,
                                  mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                  )

        # Draw right hand landmarks
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                                  )

        # Draw left hand landmarks
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                                  )

        # Draw pose detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        # Display rep count on the image
        cv2.putText(image, 'Reps: {}'.format(counter), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow('Raw Webcam Feed', image)  # Rendering our results to the screen

        if cv2.waitKey(10) & 0xFF == ord('q'):  # How we close our webcam
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\91888\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
